In [5]:
# 이미지 자동 저장

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import urllib.request
import time
import os

current_dir = os.getcwd()

f_dir = input(f'사진을 저장할 폴더를 지정하세요(예: {current_dir}\\) : ')
if f_dir == '':
    f_dir = current_dir  

query_txt = '사진저장'

save_dir = os.path.join(f_dir, query_txt)
os.makedirs(save_dir, exist_ok=True)

path = os.path.join(current_dir, "chromedriver.exe")  
service = Service(path)  
driver = webdriver.Chrome(service=service)  

s_time = time.time()      

driver.get("https://korean.visitkorea.or.kr/detail/rem_detail.html?cotid=be3db10c-b642-409c-81cc-c4cdecb5bd8b&temp=")
time.sleep(2)

def scroll_down(driver, repeat=3, pause_time=1):
    for _ in range(repeat):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
    
scroll_down(driver, repeat=5, pause_time=2)

file_no = 0                                
img_src2 = []

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img_src = soup.find('div', 'box_txtPhoto').find_all('img')

for i in img_src:
    img_src1 = i.get('src')
    if img_src1:
        img_src2.append(img_src1)

for i in range(len(img_src2)):
    try:
        urllib.request.urlretrieve(img_src2[i], os.path.join(save_dir, str(file_no) + '.jpg'))
        file_no += 1
        time.sleep(0.5)      
        print(f"{file_no} 번째 이미지 저장중입니다=======")
    except Exception as e:
        print(f"이미지 저장 중 오류 발생: {e}")
        continue

e_time = time.time()
t_time = e_time - s_time

print("=" * 70)
print(f"총 소요시간은 {round(t_time, 1)} 초 입니다.")
print(f"총 저장 건수는 {file_no} 건 입니다.")
print(f"파일 저장 경로: {save_dir} 입니다.")
print("=" * 70)

driver.close()


사진을 저장할 폴더를 지정하세요(예: C:\Users\jun05\pythonLec\) :  


1 번째 이미지 저장중입니다=======
2 번째 이미지 저장중입니다=======
3 번째 이미지 저장중입니다=======
4 번째 이미지 저장중입니다=======
5 번째 이미지 저장중입니다=======
6 번째 이미지 저장중입니다=======
7 번째 이미지 저장중입니다=======
8 번째 이미지 저장중입니다=======
9 번째 이미지 저장중입니다=======
10 번째 이미지 저장중입니다=======
11 번째 이미지 저장중입니다=======
12 번째 이미지 저장중입니다=======
13 번째 이미지 저장중입니다=======
14 번째 이미지 저장중입니다=======
총 소요시간은 20.2 초 입니다.
총 저장 건수는 14 건 입니다.
파일 저장 경로: C:\Users\jun05\pythonLec\사진저장 입니다.


In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import urllib.request
import time
import os
import math

print("=" * 80)
print(" pixabay 사이트에서 이미지를 검색하여 수집하는 크롤러 입니다 ")
print("=" * 80)

query_txt = input('1. 크롤링할 이미지의 키워드는 무엇입니까?: ')
cnt = int(input('2. 크롤링 할 건수는 몇 건입니까?: '))
real_cnt = math.ceil(cnt / 100)
f_dir = input('3. 파일이 저장될 경로만 쓰세요(예: c:\\temp\\ ) : ')

if f_dir == '':
    f_dir = os.getcwd()

save_dir = os.path.join(f_dir, query_txt)
os.makedirs(save_dir, exist_ok=True)

print("\n요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^\n")

path = os.path.join(os.getcwd(), "chromedriver.exe")
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get('https://pixabay.com/ko/')
time.sleep(3)

search_box = driver.find_element(By.NAME, 'search')  # 검색창 요소 찾기
search_box.send_keys(query_txt)
search_box.submit()

file_no = 1
count = 1
img_src2 = set() 

opener = urllib.request.build_opener()
opener.addheaders = [
    ('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36')
]
urllib.request.install_opener(opener)

def scroll_down(driver):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

s_time = time.time()

for a in range(1, real_cnt + 1):
    for _ in range(5): 
        scroll_down(driver)
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    img_src = soup.find_all('img', src=True)
    
    for c in img_src:
        img_src1 = c['src'] 
        
        if img_src1.startswith('https://cdn.pixabay.com'):
            img_src2.add(img_src1)
            print(f"수집한 이미지 URL: {img_src1}")
            
        count += 1
        if count > cnt:
            break

    if file_no >= cnt:
        break

for img_url in img_src2:
    try:
        file_path = os.path.join(save_dir, f"{file_no}.jpg")
        for attempt in range(3):
            try:
                urllib.request.urlretrieve(img_url, file_path)
                print(f"{file_no} 번째 이미지 저장 중입니다=======")
                file_no += 1
                time.sleep(0.5)
                break
            except Exception as e:
                print(f"이미지 저장 중 오류 발생 (재시도 {attempt + 1}): {e}")
                if attempt == 2:  
                    raise e
    except Exception as e:
        print(f"이미지 저장 중 최종 실패: {e}")
        continue

e_time = time.time()
t_time = e_time - s_time

print("=" * 70)
print(f"총 소요시간은 {round(t_time, 1)} 초 입니다.")
print(f"총 저장 건수는 {file_no - 1} 건 입니다.")
print(f"파일 저장 경로: {save_dir} 입니다.")
print("=" * 70)

driver.close()


 pixabay 사이트에서 이미지를 검색하여 수집하는 크롤러 입니다 


1. 크롤링할 이미지의 키워드는 무엇입니까?:  고양이
2. 크롤링 할 건수는 몇 건입니까?:  20
3. 파일이 저장될 경로만 쓰세요(예: c:\temp\ ) :  



요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^

수집한 이미지 URL: https://cdn.pixabay.com/photo/2014/11/30/14/11/cat-551554_1280.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2023/07/30/00/12/cat-8157889_640.png
수집한 이미지 URL: https://cdn.pixabay.com/photo/2018/10/01/09/21/pets-3715733_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2015/11/16/14/43/cat-1045782_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2015/03/27/13/16/maine-coon-694730_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2024/02/17/00/18/cat-8578562_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2017/11/09/21/41/cat-2934720_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2014/04/13/20/49/cat-323262_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2021/10/27/19/09/cat-6748193_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2016/09/05/21/37/cat-1647775_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2017/08/07/16/36/cat-2605502_640.jpg
수집한 이미지 URL: https://cdn.pixabay.com/photo/2022/10/25/04/55/cat-7544821_640.jp